In [11]:
input="""Step J must be finished before step H can begin.
Step N must be finished before step C can begin.
Step G must be finished before step P can begin.
Step M must be finished before step I can begin.
Step H must be finished before step X can begin.
Step B must be finished before step Y can begin.
Step C must be finished before step L can begin.
Step F must be finished before step I can begin.
Step V must be finished before step O can begin.
Step W must be finished before step Q can begin.
Step E must be finished before step L can begin.
Step U must be finished before step S can begin.
Step D must be finished before step K can begin.
Step Y must be finished before step X can begin.
Step T must be finished before step R can begin.
Step I must be finished before step K can begin.
Step A must be finished before step K can begin.
Step L must be finished before step X can begin.
Step Q must be finished before step S can begin.
Step S must be finished before step O can begin.
Step P must be finished before step Z can begin.
Step X must be finished before step R can begin.
Step Z must be finished before step O can begin.
Step O must be finished before step K can begin.
Step R must be finished before step K can begin.
Step J must be finished before step W can begin.
Step F must be finished before step V can begin.
Step A must be finished before step X can begin.
Step Z must be finished before step K can begin.
Step M must be finished before step O can begin.
Step X must be finished before step K can begin.
Step E must be finished before step K can begin.
Step J must be finished before step K can begin.
Step E must be finished before step Y can begin.
Step B must be finished before step Q can begin.
Step X must be finished before step Z can begin.
Step D must be finished before step L can begin.
Step N must be finished before step I can begin.
Step N must be finished before step B can begin.
Step V must be finished before step A can begin.
Step H must be finished before step R can begin.
Step N must be finished before step L can begin.
Step U must be finished before step O can begin.
Step A must be finished before step O can begin.
Step V must be finished before step Z can begin.
Step O must be finished before step R can begin.
Step Q must be finished before step P can begin.
Step F must be finished before step Q can begin.
Step P must be finished before step R can begin.
Step S must be finished before step X can begin.
Step J must be finished before step E can begin.
Step V must be finished before step P can begin.
Step M must be finished before step D can begin.
Step I must be finished before step S can begin.
Step Q must be finished before step O can begin.
Step M must be finished before step H can begin.
Step W must be finished before step X can begin.
Step D must be finished before step O can begin.
Step X must be finished before step O can begin.
Step Y must be finished before step Z can begin.
Step F must be finished before step L can begin.
Step V must be finished before step T can begin.
Step V must be finished before step E can begin.
Step Y must be finished before step A can begin.
Step I must be finished before step R can begin.
Step L must be finished before step O can begin.
Step U must be finished before step X can begin.
Step Q must be finished before step X can begin.
Step P must be finished before step X can begin.
Step G must be finished before step C can begin.
Step A must be finished before step L can begin.
Step M must be finished before step U can begin.
Step L must be finished before step S can begin.
Step S must be finished before step P can begin.
Step S must be finished before step K can begin.
Step F must be finished before step T can begin.
Step Q must be finished before step K can begin.
Step G must be finished before step M can begin.
Step G must be finished before step F can begin.
Step T must be finished before step Q can begin.
Step F must be finished before step Z can begin.
Step I must be finished before step Z can begin.
Step N must be finished before step X can begin.
Step J must be finished before step F can begin.
Step W must be finished before step E can begin.
Step M must be finished before step Z can begin.
Step G must be finished before step X can begin.
Step V must be finished before step U can begin.
Step P must be finished before step O can begin.
Step U must be finished before step R can begin.
Step G must be finished before step Z can begin.
Step F must be finished before step R can begin.
Step L must be finished before step R can begin.
Step F must be finished before step A can begin.
Step I must be finished before step O can begin.
Step D must be finished before step T can begin.
Step U must be finished before step L can begin.
Step B must be finished before step S can begin.
Step S must be finished before step Z can begin.
Step J must be finished before step N can begin.
Step H must be finished before step T can begin."""

from heapq import *
from collections import defaultdict

def setup(input):
    deps = defaultdict(lambda: set()) # key reqs value
    inv_deps = defaultdict(lambda: set()) # key reqd by value
    for line in input.split("\n"):
        deps[line[36]].add(line[5])
        inv_deps[line[5]].add(line[36])
    
    return deps, inv_deps

In [7]:
def execute_next():
    try:
        xeqt = heappop(executable)
    except IndexError:
        return # assuming no unresolvable dependencies exist
            
    yield xeqt
    for step in inv_deps[xeqt]:
        deps[step].remove(xeqt)
        if not deps[step]:
            heappush(executable, step)
        
    yield from execute_next()

In [16]:
def time1():
    deps, inv_deps = setup(input)
    executable = list(inv_deps.keys() - deps.keys())
    heapify(executable)
    return "".join(x for x in execute_next())

print(time1())
%timeit time1()

deps, inv_deps = setup(input)
executable = list(inv_deps.keys() - deps.keys())
heapify(executable)
"".join(x for x in execute_next())


54.6 µs ± 363 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


'GJFMDHNBCIVTUWEQYALSPXZORK'

In [4]:
req_time = lambda task: 60 + ord(task) - ord('A') + 1

class Execution2(object):
    def __init__(self, input, workers=5):
        self.finished = False
        self.time = 0
        self.workers = workers
        self.in_progress = []

        self.deps, self.inv_deps = setup(input)
        self.executable = list(self.inv_deps.keys() - self.deps.keys())
        
    def start_task(self):
        if self.workers == 0:
            yield self.complete_task()
        while not self.executable:
            yield self.complete_task()

        task = heappop(self.executable)
        self.workers -= 1
        heappush(self.in_progress, (self.time + req_time(task), task))

    def complete_task(self):
        time, task = heappop(self.in_progress)    
        self.time = time
        self.workers += 1

        for step in self.inv_deps[task]:
            self.deps[step].remove(task)
            if not self.deps[step]:
                heappush(self.executable, step)
        return task

    def run(self):
        try:
            yield from self.start_task()
        except IndexError:  # complete_task will give an indexerror
            return
        yield from self.run()

In [20]:
def time2():
    xqtr = Execution2(input)
    "".join(t for t in xqtr.run())
    return xqtr.time

%timeit time2()
print(time2())

169 µs ± 3.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1050
